NOTE:
Put the notebook or script in the same directory as hdf5 files. For routinary testing, skip find shift, read shift corrections, create panorama (unless you want to see stitched projections), and manual center refining.

In [2]:
import tomosaic
import tomopy
import dxchange
import numpy as np
import h5py
from scipy.ndimage import imread
import os, time, sys
import shutil
from itertools import izip
import glob
from scipy.misc import imread

In [ ]:
# -------- options --------
blend_method = 'pyramid'
pr_method = 'paganin'
ds_level = 2
pr_kwargs = {'distance': 50,
             'pixel': 0.8e-4 * ds_level,
             'energy': 25,
             'alpha': 1e-3}
dtype = 'float16'
# --------- paths ---------
if 'data_raw' in os.getcwd():
    print('Warning: the current directory may not be the root folder. The script must be executed in the root '
          'directory, i.e., outside of any data_raw_xx folders!')
raw_folder = os.getcwd()
moved_data_folder = os.getcwd() + '/data_raw_1x'
ds_data_folder = os.getcwd() + '/data_raw_{:d}x'.format(ds_level)
fuse_folder = os.getcwd() + '/fulldata_flatcorr_{:d}x'.format(ds_level)
paras_folder = os.getcwd()
raw_save_folder = os.getcwd() + '/fulldata_raw_{:d}x'.format(ds_level)
pr_save_folder = os.getcwd() + '/fulldata_{:s}_{:d}x'.format(pr_method, ds_level)
recon_raw_save_folder = os.getcwd() + '/recon_raw_{:d}x'.format(ds_level)
recon_pr_save_folder = os.getcwd() + '/recon_{:s}_{:d}x'.format(pr_method, ds_level)
# --------- metas ---------
prefix = 'eshrew_Os_stand_mosaic_70ms_25739eV_300mmWD_10X3_0'
x_cam = np.floor(1920 / ds_level)
y_cam = np.floor(1200 / ds_level)
x_shift = np.floor(1572 / ds_level)
y_shift = np.floor(1109 / ds_level)
center_guess = 6440 / ds_level
center_vec = np.array([6451, 6447, 6440, 6427, 6424, 6421, 6420, 6414, 6404, 6399, 6399])
center_vec /= ds_level

In [ ]:
# try root folder, then ./data_raw_1x/
try:
    file_list = tomosaic.get_files(raw_folder, prefix, type='h5')
    file_grid = tomosaic.start_file_grid(file_list, pattern=1)
    shift_grid = tomosaic.start_shift_grid(file_grid, x_shift, y_shift)
    f = h5py.File(file_list[0], 'r')
    full_shape = f["exchange/data"].shape
    n_frames = full_shape[0]
    os.chdir(raw_folder)
except:
    try:
        file_list = tomosaic.get_files(moved_data_folder, prefix, type='h5')
        file_grid = tomosaic.start_file_grid(file_list, pattern=1)
        f = h5py.File(moved_data_folder+'/'+file_list[0], 'r')
        full_shape = f["exchange/data"].shape
        n_frames = full_shape[0]
        os.chdir(raw_folder)
    except:
        print('Error: no file exists in root folder. If hdf5\'s has been moved to elsewhere, update file grid.')
        exit()

In [ ]:
os.chdir(raw_folder)
if os.path.exists('data_raw_1x'):
    for f in file_list:
        shutil.move(f, 'data_raw_1x/' + f)

In [ ]:
os.chdir(raw_folder)
tomosaic.util.reorganize_dir(file_list, raw_ds=[2, 4], pr_ds=[])

In [ ]:
os.chdir(moved_data_folder)
for (y, x), value in np.ndenumerate(file_grid):
    try:
        prj, flt, drk = dxchange.read_aps_32id(moved_data_folder+'/'+value, proj=(0, 1))
        fname = prefix + 'Y' + str(y).zfill(2) + '_X' + str(x).zfill(2)
        dxchange.write_tiff_stack(flt, fname=os.path.join('partial_frames', 'partial_flats', fname))
        dxchange.write_tiff_stack(drk, fname=os.path.join('partial_frames', 'partial_darks', fname))
    except:
        print('({:d}, {:d} not written.)'.format(y, x))
os.chdir(raw_folder)

In [ ]:
os.chdir(moved_data_folder)
refined_shift = tomosaic.refine_shift_grid(file_grid, shift_grid)
np.savetxt(paras_folder+"/shifts.txt", refined_shift, fmt=str('%4.2f'))
os.chdir(raw_folder)

In [ ]:
os.chdir(raw_folder)
shift_grid = tomosaic.util.file2grid(paras_folder+"/shifts.txt")
shift_grid = tomosaic.absolute_shift_grid(shift_grid, file_grid)
print shift_grid

In [ ]:
shift_grid /= ds_level

In [ ]:
# --------------------------------#
# Set ds_level before proceeding. #
# --------------------------------#
loc = raw_input('Do you want to stitch 1x or downsampled data? (1 or d)')
if loc == '1':
    os.chdir(moved_data_folder)
elif loc == 'd':
    os.chdir(ds_data_folder)
else:
    print('Error: check your input.')
    exit()
t0 = time.time()
print "    Building panograms..."
for frame in range(0, n_frames, int(n_frames/4)-1):
    print "        Now at " + str(frame) + "/" + str(n_frames)
    pano = tomosaic.build_panorama(file_grid, shift_grid, frame=frame, method='pyramid').astype('float32')
    dxchange.write_tiff(pano, "panos_{:d}x/frame{:04d}".format(ds_level, frame))
print "    Building done in " + str(time.time() - t0) + " sec."
os.chdir(raw_folder)

In [ ]:
# --------------------------------#
# Set ds_level before proceeding. #
# --------------------------------#
# create h5 for raw data without PR
tomosaic.util.total_fusion(ds_data_folder, fuse_folder, 'fulldata_flatcorr.h5', file_grid, shift_grid, 
                           blend_method=blend_method)

In [ ]:
os.chdir(moved_data_folder)
for center in range(center_guess - 20, center_guess + 20, 1):
    center_vec[:] = center
    tomosaic.recon.recon_block(file_grid, shift_grid, (y_cam/2, y_cam/2+(file_grid.shape[0]-1)*y_shift+10), 
                               y_shift, raw_folder+"/center_test", center_vec, algorithm='gridrec')
os.chdir(raw_folder)

In [ ]:
# --------------------------------#
# Set ds_level before proceeding. #
# --------------------------------#
loc = raw_input('Do you want to stitch 1x or downsampled data? (1 or d)')
if loc == '1':
    os.chdir(moved_data_folder)
elif loc == 'd':
    os.chdir(ds_data_folder)
else:
    print('Error: check your input.')
    exit()
tomosaic.recon.recon_block(file_grid, shift_grid, (0, 6000), 1, folder + "/recon", center_vec, 
                                    blend_method='pyramid', algorithm='gridrec', filter_name='parzen', 
                                    format='float32', crop=[[1300, 0], [9000, 12000]])

In [ ]:
# -------------------------------- #
# Set ds_level before proceeding.  #
# -------------------------------- # 
start = 0
end = 6000
tp = raw_input('Use phase-retrieved data? (y/n)')
if tp == 'y':
    fname = 'fulldata_pag_{:d}x.h5'.format(ds_level)
    os.chdir(pr_save_folder)
elif tp == 'n':
    fname = 'fulldata_raw_{:d}x.h5'.format(ds_level)
    os.chdir(raw_save_folder)
theta = tomopy.angles(n_frames)
grid_bins = np.ceil(shift_grid[:, 0])
for slice in range(start, end):
    proj, _, _ = dxchange.read_aps_32id(fname, sino=(slice, slice+1))
    grid_line = np.digitize(slice, grid_bins)
    grid_line = grid_line[0]
    center = center_vec[grid_line]
    rec = tomopy.recon(proj, theta, center=center, algorithm='gridrec')
    rec = tomopy.circ_mask(rec, axis=0, ratio=0.95)
    dxchange.write_tiff(rec, fname='recon_{:d}x_{:05d}'.format(ds_level, slice), dtype='float16')
os.chdir(raw_folder)

In [ ]:
tp = raw_input('Use phase-retrieved data? (y/n)')
if tp == 'y':
    os.chdir(recon_pr_save_folder)
    fname = 'fulldata_pag_{:d}x.h5'.format(ds_level)
    f = h5py.File('recon_pag_{:d}x'.format(ds_level))
elif tp == 'n':
    os.chdir(recon_raw_save_folder)
    fname = 'fulldata_raw_{:d}x.h5'.format(ds_level)
    os.chdir(raw_save_folder)
    f = h5py.File('recon_raw_{:d}x'.format(ds_level))
tiff_list = glob.glob('*.tiff').sorted()
temp = imread(tiff_list[0])
dset = f.create_dataset('data', (len(tiff_list), temp.shape[0], temp.shape[1]))
for i in range(len(tiff_list)):
    dset[i, :, :] = imread(tiff_list[i])
f.close()
os.chdir(raw_folder)